In [32]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

#LANGSMITH TRACING

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHIAN_TRACING_V2'] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [33]:
# Data ingestion --->  From website--> scraping the data

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://medium.com/around-the-prompt/what-is-langchain-and-why-should-i-care-as-a-developer-b2d952c42b28")       #want to take url from user on streamlit-- chatgppt command

docs = loader.load()

docs

[Document(metadata={'source': 'https://medium.com/around-the-prompt/what-is-langchain-and-why-should-i-care-as-a-developer-b2d952c42b28', 'title': 'What is Langchain and why should I care as a developer? | by Logan Kilpatrick | Around the Prompt | Medium', 'description': 'Langchain 🦜 is one of the fastest growing open source projects in history, in large part due to the explosion of interest in LLM’s. This post explores some of the cool thing that langchain helps…', 'language': 'en'}, page_content='What is Langchain and why should I care as a developer? | by Logan Kilpatrick | Around the Prompt | MediumOpen in appSign upSign inWriteSign upSign inWhat is Langchain and why should I care as a developer?The worlds fastest growing language model application framework, packed with LLM tools and agent supportLogan Kilpatrick·FollowPublished inAround the Prompt·6 min read·Jun 30, 2023--4ListenShareLangchain 🦜 is one of the fastest growing open source projects in history, in large part due to t

In [34]:
#data splitting of docs

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://medium.com/around-the-prompt/what-is-langchain-and-why-should-i-care-as-a-developer-b2d952c42b28', 'title': 'What is Langchain and why should I care as a developer? | by Logan Kilpatrick | Around the Prompt | Medium', 'description': 'Langchain 🦜 is one of the fastest growing open source projects in history, in large part due to the explosion of interest in LLM’s. This post explores some of the cool thing that langchain helps…', 'language': 'en'}, page_content='What is Langchain and why should I care as a developer? | by Logan Kilpatrick | Around the Prompt | MediumOpen in appSign upSign inWriteSign upSign inWhat is Langchain and why should I care as a developer?The worlds fastest growing language model application framework, packed with LLM tools and agent supportLogan Kilpatrick·FollowPublished inAround the Prompt·6 min read·Jun 30, 2023--4ListenShareLangchain 🦜 is one of the fastest growing open source projects in history, in large part due to t

In [35]:
#converting the text vector to the embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [36]:
#storing the embeddings into a vector database

from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [37]:
vectorstoredb

first approch where we give context and query manually

In [38]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o")

In [39]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000273DBA9CEB0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000273DBA9FFD0>, root_client=<openai.OpenAI object at 0x00000273DBA9C6D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000273DBA9CF10>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [47]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on the provided context, which is easy for user to understand:
    <context>
    {context}
    </context>
    
    """

)

document_chain = create_stuff_documents_chain(llm,prompt)

document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context, which is easy for user to understand:\n    <context>\n    {context}\n    </context>\n    \n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000273DBA9CEB0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000273DBA9FFD0>, root_client=<openai.OpenAI object at 0x00000273DBA9C6D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000273DBA9CF10>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrO

In [48]:
"""from langchain_core.documents import Document

document_chain.invoke({
    "input":"what are Langsmith two usage limits",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})
"""

'from langchain_core.documents import Document\n\ndocument_chain.invoke({\n    "input":"what are Langsmith two usage limits",\n    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we\'ve been tracking on our usage graph. ")]\n})\n'

Dynamically select the relevent document and pass for the input provided by the user

In [49]:
vectorstoredb

In [50]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever, document_chain)
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000273DBA9C7F0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based on the provided context, which is easy for user to understand:\n    <context>\n    {context}\n    </context

In [51]:
#Getting the response

response = retriever_chain.invoke({"input":"what is langchain"})

In [52]:
response['answer']

"Langchain is an open-source framework that helps developers work with large language models (LLMs) by providing a flexible and simplified toolset. It allows you to connect data with models like OpenAI's GPT, create agent workflows, and test multiple LLM offerings seamlessly. By using Langchain, developers can manage tasks such as prompt chaining, logging, and connecting to data sources more efficiently. As a developer, Langchain is valuable because it offers a single interface to manage different model technologies, boasting a large community that supports its growth and ensures its long-term viability. Additionally, with its agent API, developers can create autonomous programmatic entities, making it possible for models to plan, execute tasks, and accomplish goals independently. The framework's popularity is reflected in its 51k stars on GitHub and over a million downloads per month, illustrating its usefulness and the trust it's earned within the developer community."